In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
from scipy.spatial.distance import euclidean
warnings.filterwarnings('ignore')
sc = SparkSession \
       .builder \
       .appName("Data") \
       .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw = pd.read_csv('/content/drive/MyDrive/Clouds/Cources/BigData/midterm/BX-Book-Ratings.csv', encoding = 'unicode_escape',error_bad_lines=False,sep=";")
raw_books = pd.read_csv('/content/drive/MyDrive/Clouds/Cources/BigData/midterm/BX-Books.csv', encoding = 'unicode_escape',error_bad_lines=False,sep=";")

In [ ]:
train = raw.sample(n=200000)

ratings_new=train

ratings_explicit = ratings_new[ratings_new['Book-Rating'] != 0]
ratings_implicit = ratings_new[ratings_new['Book-Rating'] == 0]

n_users = ratings_explicit['User-ID'].unique().shape[0]
n_books= ratings_explicit['ISBN'].unique().shape[0]
## 1
print(n_users, n_books)
## 2
print(ratings_explicit.shape)

counts1 = ratings_explicit['User-ID'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['User-ID'].isin(counts1[counts1 >= 20].index)]
counts = ratings_explicit['Book-Rating'].value_counts()
## 3
print(ratings_explicit.shape)

# ratings_matrix = ratings_explicit.pivot(index='ISBN', columns='User-ID', values='Book-Rating')
# ISBN = ratings_matrix.index
# userID = ratings_matrix.columns
## 5
# print(ratings_matrix.shape)

# ratings_matrix.fillna(0, inplace = True)
# ratings_matrix = ratings_matrix.astype(np.int32)
## 7

26730 49430
(75164, 3)
(30103, 3)


In [ ]:
import pyspark.pandas as ps
ratings_matrix = ps.from_pandas(ratings_explicit)
ratings_matrix = ratings_matrix.pivot(index='ISBN', columns='User-ID', values='Book-Rating')
# ratings_matrix.dropna(axis='columns', how='all')
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
ratings_matrix.shape

In [ ]:
def item_based_book_recomender(n_recommend=10):
  n = 0
  while n < n_recommend:
    book = ratings_matrix.iloc[n].values
    df3 = ratings_matrix.apply(lambda x: euclidean(x.values, book), axis=1)
    similar_books = df3.sort_values(ascending=True)[0:5].index.values
    print(f"Nếu người đọc thích cuốn {raw_books.loc[raw_books['ISBN'] == ratings_matrix.iloc[n,:].name]['Book-Title'].values[0]}, gợi ý cho người đọc những cuốn sau:")
    for i in similar_books:
      try:
        print("\t- " + str(raw_books.loc[raw_books['ISBN'] == i]['Book-Title'].values[0]))
      except:
        print('Mã sách bị lỗi')
    print('\n-----------')
    n += 1

In [ ]:
import time
start = time.time()
item_based_book_recomender(3)
start - time.time()

In [ ]:
# ratings_matrix1 = ratings_explicit
# ratings_matrix1 = ratings_matrix1.pivot(index='ISBN', columns='User-ID', values='Book-Rating')
# # ratings_matrix.dropna(axis='columns', how='all')
# ratings_matrix1.fillna(0, inplace = True)
# ratings_matrix1 = ratings_matrix.astype(np.int32)
# ratings_matrix1.shape

In [ ]:
# def item_based_book_recomender_no_spark(n_recommend=10):
#   n = 0
#   while n < n_recommend:
#     book = ratings_matrix1.iloc[n,:].values
#     df3 = ratings_matrix1.apply(lambda x: euclidean(x.values, book), axis=1)
#     similar_books = df3.sort_values(ascending=True)[0:5].index.values
#     print(f"Nếu người đọc thích cuốn {raw_books.loc[raw_books['ISBN'] == ratings_matrix1.iloc[n,:].name]['Book-Title'].values[0]}, gợi ý cho người đọc những cuốn sau:")
#     for i in similar_books:
#       try:
#         print("\t- " + str(raw_books.loc[raw_books['ISBN'] == i]['Book-Title'].values[0]))
#       except:
#         print('Mã sách bị lỗi')
#     print('\n-----------')
#     n += 1

In [ ]:
# start = time.time()
# item_based_book_recomender_no_spark(3)
# start - time.time()